# ADS Coursework 2023 - Geographic Sentiment Distribution of Trending Social Media Topics 

## Initial Setup - Twitter

In [5]:
import re
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
from geopy.geocoders import Nominatim
from functools import partial

keywords = "(AI, Artificial Intelligence, artificial intelligence, ai) -filter:links -ad -free -purchase -premium -avail -claim -giveaway -participants -Telegram -winner -win -credits -token -tokens -football -aiART -artwork -art -cosplay -character -waifu -generated lang:en"
start_date = dt.datetime(2017, 1, 1) # Start date
end_date = dt.datetime(2019, 12, 31) # End date
limit = 10 # Number of tweets per month

queries = []
while start_date < end_date:
    month_start = start_date
    month_end = start_date + relativedelta(months=+1) - timedelta(days=1)
    if month_end > end_date:
        month_end = end_date
    
    query = f'{keywords} until:{month_end.date()} since:{month_start.date()}'
    queries.append(query)
    
    start_date = month_end + timedelta(days=1)

In [9]:
pattern = r"^\w+,\s\w+$"

tweets = []
tweets_content = []
count = 0
for query in tqdm(queries): 
    for tweet in sntwitter.TwitterSearchScraper(query, top = True).get_items():
        if  count == limit:
            count = 0
            break
        if  tweet.user.location != '':
            # Use regular expression to match the location pattern
            match = re.match(pattern, tweet.user.location)
            if match:
                tweet.user.location = match.group()
                date_str = tweet.date.strftime('%Y-%m-%d')
                nA_str = "N/A"
                twitter_str = "Twitter"
                tweets.append([tweet.user.username, date_str, tweet.content, tweet.likeCount, tweet.id, tweet.replyCount, nA_str, nA_str, twitter_str, tweet.user.location])
                count += 1

    columns = ['User', 'Date', 'Content', 'Reactions', 'ID', 'N_Children', 'Post Title', 'Subreddit', 'Platform', 'Location']

df = pd.DataFrame(tweets, columns=columns)
df

  0%|                                                                                           | 0/36 [00:00<?, ?it/s]C:\Users\arafa\AppData\Local\Temp\ipykernel_9836\3159114692.py:7: DeprecatedFeatureWarning: `top` argument is deprecated, use `mode = snscrape.modules.twitter.TwitterSearchScraperMode.TOP` instead of `top = True`
  for tweet in sntwitter.TwitterSearchScraper(query, top = True).get_items():
C:\Users\arafa\AppData\Local\Temp\ipykernel_9836\3159114692.py:19: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.user.username, date_str, tweet.content, tweet.likeCount, tweet.id, tweet.replyCount, nA_str, nA_str, twitter_str, tweet.user.location])
100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [02:19<00:00,  3.89s/it]


,User,Date,Content,Reactions,ID,N_Children,Post Title,Subreddit,Platform,Location
0,Aiman_9560,2017-01-21,@satyanadella @Microsoft @Google Does the AI(...,1,822787128170074112,0,N/A,N/A,Twitter,"Kharagpur, India"
1,SwiftOnSecurity,2017-01-08,"""Artificial Intelligence"" is an ultimately dis...",71,818211529703178242,8,N/A,N/A,Twitter,"Cyber, USA"
2,shopperann68,2017-01-08,@Quad_Finn @petestavros @SeaWorld @dodo This g...,17,818161290271137793,1,N/A,N/A,Twitter,"Arizona, USA"
3,SaraKubik,2017-01-22,"I love tech, but to me, #AI first meant Adobe ...",7,823023339706925057,1,N/A,N/A,Twitter,"Chicago, IL"
4,JoseyMras,2017-01-28,The best way to succeed in this internet centu...,6,825411883616329728,0,N/A,N/A,Twitter,"Nakuru, Kenya"
...,...,...,...,...,...,...,...,...,...,...
355,neil_sahota,2019-12-03,"If you had only one wish to help, and it had t...",3,1201844443709620225,2,N/A,N/A,Twitter,"Irvine, CA"
356,berkanttubi,2019-12-22,AI : Artificial Intelligence for those who don...,0,1208879575947776000,1,N/A,N/A,Twitter,"Ankara, Türkiye"
357,KLAATU777,2019-12-23,@GeorgeTakei You forgot AI Artificial Intellig...,0,1208927593669570560,0,N/A,N/A,Twitter,"Columbus, OH"
358,desm7496,2019-12-15,a machine disconnected from the human emotion ...,11,1206025028128661504,1,N/A,N/A,Twitter,"Michigan, USA"


In [15]:
geolocator = Nominatim(user_agent="coursework2023Notebook")

for location in df.Location:
    geocode = partial(geolocator.geocode, language="en")
    address = geocode(location)
    if address is not None:
        address_str = address.address
        address_list = address_str.split(',')
        country = address_list[-1].strip()
    location = country

Location not found
Location not found
Location not found


In [33]:
# to save to csv
df.to_csv('tweets.csv')

In [4]:
# load model and tokeniser
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
labels = ['Negative', 'Neutral', 'Positive']

In [14]:
highest_labels = []

for tweet in tweets_content:

    # preprocess tweet
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'

        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)

    # sentiment analysis
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    highest_label = 'None'
    highest_score = 0
    
    print(tweet)

    for i in range(len(scores)):

        l = labels[i]
        s = scores[i]
        if s > highest_score:
            highest_label = l
            highest_score = s
        print(l, s)
        
    highest_labels.append(highest_label)
    
    print('\n')

@JudgeCarter There have been tremendous advancement in AI (Artificial Intelligence), robotics and autonomous vehicles which is causing a great deal of concern for the people I talk to. They are afraid they will lose their jobs &amp; their children &amp;grandchildren won't have any how will they live?
Negative 0.46596152
Neutral 0.407507
Positive 0.12653144


@AhmadGanga The rise of Artificial intelligence(AI) is a big threat to humanity. We gonna lose our concept of humanity to algorithms.
Negative 0.7944987
Neutral 0.18753004
Positive 0.01797127


I was born and raised in #Germany. This country and its people always fascinate me in a certain way. When it comes down to it, they can function efficiently like a sophisticated machine.  So efficiently that even Artificial Intelligence is scared shitless.

#AI #CoronaCrisis
Negative 0.54484105
Neutral 0.30624062
Positive 0.14891835


@BillGates I disrespectfully decline your toxic vaccines. They haven't been tested, what are the ingredients

Artificial intelligence presents the most potent and holistic solution class to respond to this pressurized investment and trading environment. 

#investment #trading #opportunities #market #investments #AI
Negative 0.00555405
Neutral 0.14019471
Positive 0.85425127


Artificial Intelligence and Agriculture

Transformative companies are innovating agriculture with AI

Read more in the September 2022 issue of the Precision Agriculture Digital Digest to learn about the innovative companies ⤵️

#PrecisionAg #AgTech #PrecisionFarming #AI #OntAg
Negative 0.007375832
Neutral 0.51804703
Positive 0.47457713


I told my friend that I wanted to learn digital marketing first, then learn full-stack web development, before going into Data Science and Artificial Intelligence (AI). Lol!  Do you know that ‘who is this guy' kind of look? That's what I got.
Negative 0.101339415
Neutral 0.57360405
Positive 0.32505658


"The true challenge to artificial intelligence proved to be solving the tasks that are 

In [8]:
print(highest_labels)

['Negative', 'Negative', 'Negative', 'Negative', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Positive', 'Neutral', 'Negative', 'Positive', 'Positive', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Neutral', 'Neutral', 'Negative', 'Positive', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Neutral', 'Neutral', 'Neutral', 'Neutral']
